<a href="https://colab.research.google.com/github/yarahi023/course-collaboration-travel-plans/blob/master/Copy_of_Image_Classifier_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available()) # Should return True when GPU is enabled.


2.3.1+cu121
True


# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications.

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below.

<img src='https://github.com/udacity/aipnd-project/blob/master/assets/Flowers.png?raw=1' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [ ]:
# Imports here
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
from torch import tensor
from torch.optim import lr_scheduler

from PIL import Image
import PIL
import numpy as np
import time
import torch
#import helper
import torch.nn as nn
from torch import nn, optim
#from torch.optim import lr_scheduler
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision import datasets, transforms, models
from collections import OrderedDict
from torch.utils.data import DataLoader
import json
#import argparse
import torchvision.models as models
#import seaborn as sns
from torchvision.models import VGG16_Weights, vgg16
from torchvision.models import vgg16, VGG16_Weights
#from torch.optim.lr_scheduler import StepLR

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.


In [ ]:
!wget 'https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz'

!mkdir flowers && tar -xzf flower_data.tar.gz -C flowers

--2024-08-17 06:49:53--  https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.226.8, 52.217.167.136, 16.182.40.176, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.226.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 344873452 (329M) [application/x-gzip]
Saving to: ‘flower_data.tar.gz’

flower_data.tar.gz  100%[===================>] 328.90M  48.5MB/s    in 7.2s    

2024-08-17 06:50:01 (45.6 MB/s) - ‘flower_data.tar.gz’ saved [344873452/344873452]



In [ ]:
!rm -rf flowers
!mkdir flowers && tar -xzf flower_data.tar.gz -C flowers


In [ ]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [ ]:
# TODO: Define your transforms for the training, validation, and testing sets
data_transforms = {
    'train' : transforms.Compose(
        [transforms.RandomResizedCrop(size=224),
        transforms.RandomRotation(30),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],
                             [0.229,0.224,0.225])
    ]),
    'validation' : transforms.Compose(
        [transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],
                             [0.229,0.224,0.225])
    ]),
    'test' : transforms.Compose(
        [ transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],
                             [0.229,0.224,0.225])
    ])
}


# TODO: Load the datasets with ImageFolder
image_datasets = {
    'train' : datasets.ImageFolder(train_dir, transform=data_transforms['train']),
    'validation' : datasets.ImageFolder(valid_dir, transform=data_transforms['validation']),
    'test' : datasets.ImageFolder(test_dir, transform=data_transforms['test'])
}


# TODO: Using the image datasets and the trainforms, define the dataloaders
dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size = 64, shuffle = True),
    'test': DataLoader(image_datasets['test'], batch_size = 64, shuffle = False),
    'validation': DataLoader(image_datasets['validation'], batch_size = 64, shuffle=False)
}

#Size of Loaders, to be used for calculating Average Loss and Accuracy
train_size = len(image_datasets['train'])
validation_size = len(image_datasets['validation'])
test_size = len(image_datasets['test'])

print(train_size, validation_size, test_size)

class_to_idx = image_datasets['train'].class_to_idx


6552 818 819


### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [ ]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
print(cat_to_name)

{'21': 'fire lily', '3': 'canterbury bells', '45': 'bolero deep blue', '1': 'pink primrose', '34': 'mexican aster', '27': 'prince of wales feathers', '7': 'moon orchid', '16': 'globe-flower', '25': 'grape hyacinth', '26': 'corn poppy', '79': 'toad lily', '39': 'siam tulip', '24': 'red ginger', '67': 'spring crocus', '35': 'alpine sea holly', '32': 'garden phlox', '10': 'globe thistle', '6': 'tiger lily', '93': 'ball moss', '33': 'love in the mist', '9': 'monkshood', '102': 'blackberry lily', '14': 'spear thistle', '19': 'balloon flower', '100': 'blanket flower', '13': 'king protea', '49': 'oxeye daisy', '15': 'yellow iris', '61': 'cautleya spicata', '31': 'carnation', '64': 'silverbush', '68': 'bearded iris', '63': 'black-eyed susan', '69': 'windflower', '62': 'japanese anemone', '20': 'giant white arum lily', '38': 'great masterwort', '4': 'sweet pea', '86': 'tree mallow', '101': 'trumpet creeper', '42': 'daffodil', '22': 'pincushion flower', '2': 'hard-leaved pocket orchid', '54': 's

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [ ]:
model = vgg16(weights=VGG16_Weights.DEFAULT)
model


In [ ]:
# TODO: Build and train your network
model = vgg16(weights=VGG16_Weights.DEFAULT)
#Freeze the convolutional layers
for param in model.features.parameters():
    param.requires_grad = False  # Freeze the convolutional layers

# Define the new classifier
classifier = nn.Sequential(OrderedDict([
    ('inputs', nn.Linear(25088, 4096)),
    ('batchnorm1', nn.BatchNorm1d(4096)),
    ('dropout', nn.Dropout(p=0.5)),
    ('relu', nn.ReLU()),
    ('fc2', nn.Linear(4096, 102)),
    ('batchnorm2', nn.BatchNorm1d(102)),
    ('output', nn.LogSoftmax(dim=1))
]))

# Replace the classifier
model.classifier = classifier

# Ensure that the classifier parameters are trainable
for param in model.classifier.parameters():
    param.requires_grad = True

# Print trainable parameters for debugging
print("Trainable parameters:")
for param in model.parameters():
    if param.requires_grad:
        print(param.shape)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()

# Initialize the optimizer
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.0001)
#scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

model.classifier = classifier

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:06<00:00, 84.7MB/s]


Trainable parameters:
torch.Size([4096, 25088])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([102, 4096])
torch.Size([102])
torch.Size([102])
torch.Size([102])


In [ ]:
epochs = 2
print_every = 25 #50 # Adjust this based on how often you want to print the stats
running_loss = 0
steps = 0
test_loss = 0
correct = 0
total = 0
for epoch in range(epochs):
    print("Epoch: {}/{}".format(epoch + 1, epochs))
    model.train()  # Set to training mode

    for images, labels in dataloaders['train']:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        steps += 1

        if steps % print_every == 0:
            model.eval()  # Set the model to evaluation mode for testing
            test_loss = 0
            correct = 0
            total = 0

        with torch.no_grad():
            for test_images, test_labels in dataloaders['test']:
                test_images, test_labels = test_images.to(device), test_labels.to(device)
                test_outputs = model(test_images)
                loss = criterion(test_outputs, test_labels)
                test_loss += loss.item()

                ps = torch.exp(test_outputs)
                top_ps, top_class = ps.topk(1, dim=1)
                equals = top_class == test_labels.view(*top_class.shape)
                correct += equals.sum().item()
                total += test_labels.size(0)

    # Calculate average test loss and accuracy
        test_loss /= len(dataloaders['test'])  # Average over the number of batches
        test_accuracy = correct / total if total > 0 else 0

        print(f"Epoch {epoch+1}/{epochs}, steps: {steps}, "
              f"Train loss: {running_loss/print_every:.4f}, "
              f"Test loss: {test_loss:.4f}, Test accuracy: {test_accuracy:.4f}")

        running_loss = 0

# Initialize validation variables
        model.eval()
        valid_loss = 0
        correct = 0
        total = 0

        model.eval()  # Set the model to evaluation mode
        with torch.no_grad():
            # Validation on the validation set
            for val_images, val_labels in dataloaders['validation']:
                val_images, val_labels = val_images.to(device), val_labels.to(device)
                val_outputs = model(val_images)
                val_loss = criterion(val_outputs, val_labels)
                valid_loss += val_loss.item()

                ps = torch.exp(val_outputs)
                top_ps, top_class = ps.topk(1, dim=1)
                equals = top_class == val_labels.view(*top_class.shape)
                correct += equals.sum().item()
                total += val_labels.size(0)

        # Calculate average validation loss and accuracy
        validation_loss = valid_loss / len(dataloaders['validation'])
        validation_accuracy = correct / total if total > 0 else 0

        print(f"Epoch {epoch+1}/{epochs}, steps: {steps}, "
              f"Train loss: {running_loss/print_every:.4f}, "
              f"Validation loss: {validation_loss:.4f}, "
              f"Validation accuracy: {validation_accuracy:.4f}")

        running_loss = 0

Epoch: 1/2
Epoch 1/2, steps: 1, Train loss: 0.1880, Test loss: 4.5211, Test accuracy: 0.0415
Epoch 1/2, steps: 1, Train loss: 0.0000, Validation loss: 4.0515, Validation accuracy: 0.1687
Epoch 1/2, steps: 2, Train loss: 0.1718, Test loss: 4.2771, Test accuracy: 0.1772
Epoch 1/2, steps: 2, Train loss: 0.0000, Validation loss: 3.8937, Validation accuracy: 0.1968
Epoch 1/2, steps: 3, Train loss: 0.1691, Test loss: 4.1261, Test accuracy: 0.1973
Epoch 1/2, steps: 3, Train loss: 0.0000, Validation loss: 3.7816, Validation accuracy: 0.2066
Epoch 1/2, steps: 4, Train loss: 0.1638, Test loss: 3.9893, Test accuracy: 0.2101
Epoch 1/2, steps: 4, Train loss: 0.0000, Validation loss: 3.6992, Validation accuracy: 0.2274
Epoch 1/2, steps: 5, Train loss: 0.1583, Test loss: 3.8733, Test accuracy: 0.2309
Epoch 1/2, steps: 5, Train loss: 0.0000, Validation loss: 3.6119, Validation accuracy: 0.2408
Epoch 1/2, steps: 6, Train loss: 0.1570, Test loss: 3.7955, Test accuracy: 0.2327
Epoch 1/2, steps: 6, Train 

In [ ]:
epochs = 3
print_every = 35
def evaluate(model, dataloader, criterion, device):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0
    correct = 0
    total = 0


    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            ps = torch.exp(outputs)
            top_ps, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            correct += equals.sum().item()
            total += labels.size(0)

    average_loss = test_loss / len(dataloader)
    accuracy = correct / total if total > 0 else 0
    return average_loss, accuracy

for epoch in range(epochs):
    print(f"Epoch: {epoch + 1}/{epochs}")
    model.train()  # Set to training mode
    running_loss = 0
    steps = 0

    for images, labels in dataloaders['train']:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        steps += 1

        if steps % print_every == 0:
            train_loss = running_loss / print_every
            test_loss, test_accuracy = evaluate(model, dataloaders['test'], criterion, device)

            print(f"Epoch {epoch+1}/{epochs}, steps: {steps}, "
                  f"Train loss: {train_loss:.4f}, "
                  f"Test loss: {test_loss:.4f}, Test accuracy: {test_accuracy:.4f}")

            running_loss = 0

    # Validation
    validation_loss, validation_accuracy = evaluate(model, dataloaders['validation'], criterion, device)
    print(f"Epoch {epoch+1}/{epochs}, "
          f"Validation loss: {validation_loss:.4f}, "
          f"Validation accuracy: {validation_accuracy:.4f}")

Epoch: 1/3
Epoch 1/3, steps: 35, Train loss: 2.5388, Test loss: 1.6387, Test accuracy: 0.8144
Epoch 1/3, steps: 70, Train loss: 1.7670, Test loss: 0.9136, Test accuracy: 0.8083
Epoch 1/3, Validation loss: 0.7089, Validation accuracy: 0.8362
Epoch: 2/3
Epoch 2/3, steps: 35, Train loss: 2.4271, Test loss: 1.7397, Test accuracy: 0.8291
Epoch 2/3, steps: 70, Train loss: 1.7673, Test loss: 0.9541, Test accuracy: 0.8205
Epoch 2/3, Validation loss: 0.6886, Validation accuracy: 0.8594
Epoch: 3/3
Epoch 3/3, steps: 35, Train loss: 2.4620, Test loss: 1.8164, Test accuracy: 0.8046
Epoch 3/3, steps: 70, Train loss: 1.7582, Test loss: 1.0162, Test accuracy: 0.8095
Epoch 3/3, Validation loss: 0.7294, Validation accuracy: 0.8692


## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# TODO: Do validation on the test set
correct,total = 0,0
with torch.no_grad():
    model.eval()
    for data in dataloaders['train']:
        images, labels = data
        images, labels = images.to('cuda'), labels.to('cuda')
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy on test images is: %d%%' % (100 * correct / total))

Accuracy on test images is: 66%


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# TODO: Save the checkpoint

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network.

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training.

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation.

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''

    # TODO: Process a PIL image for use in a PyTorch model

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()

    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))

    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean

    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)

    ax.imshow(image)

    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''

    # TODO: Implement the code to predict the class from an image file

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='https://github.com/udacity/aipnd-project/blob/master/assets/inference_example.png?raw=1' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes